In [4]:
#install the required libraries (SpeechRecognition, PyAudio)
##pip install SpeechRecognition
##pip install PyAudio
#pip install pocketsphinx #sphinx for recognizing English 
#Reason why I am trying this:
#Mark Zukerberg on CNN, he built Jarvis for himself.
#And seeing this I wanted my own AI too

SyntaxError: invalid syntax (Temp/ipykernel_27308/4044473456.py, line 4)

In [5]:
#setting up the libraries
import speech_recognition as sr


SyntaxError: invalid syntax (Temp/ipykernel_27308/3526946866.py, line 3)

In [7]:
import pocketsphinx

ModuleNotFoundError: No module named 'pocketsphinx'

In [3]:
#Trial Codes without any OpenAPIs
#Get Audio Input
r = sr.Recognizer()

#Ask language preference (I will mainly code for Korean)
print("어떤 언어로 할까요? 영어는 0, 한글은 1을 부탁드립니다")
print("What Language do you prefer? input 0 for English, 1 for Korean")
language = input()
with sr.Microphone() as source:
    if language == '0':
        print("Hi, I am Cloud One")
    else:
        print("안녕하세요, 저는 '클라우드 원'입니다")
    audio = r.listen(source)

#recognize speech using sphinx
if language == 0:
    try:
        print("I've heard " + r.recgonize_sphinx(audio))
    except sr.UnknownValueError:
        print("Sorry, I couldn't understand. I wish I could have been a better help")
    except sr.RequestError:
        print("Sorry, that is out of my ability")
else:
    try:
        print(r.recgonize_sphinx(audio)+"라고 인식했습니다")
    except sr.UnknownValueError:
        print("미안해요, 제가 좀 더 공부하고 올게요. 다음에는 꼭 도움이 될수 있는 클라우드 원이 되겠습니다")
    except sr.RequestError:
        print("죄송해요, 아직 저는 그걸 해낼 능력이 부족하네요. 언젠가는 수행할 수 있도록 최선을 다해 공부하는 클라우드 원이 되겠습니다")

#write audio to a WAV file
with open("microphone-results.wav", "wb") as f:
    f.write(audio.get_wav_data())

#To use the service better, set up your "MY application Settings">"Recordings">"Activate" to be always on

어떤 언어로 할까요? 영어는 0, 한글은 1을 부탁드립니다
What Language do you prefer? input 0 for English, 1 for Korean
0
Hi, I am Cloud One


AttributeError: 'Recognizer' object has no attribute 'recgonize_sphinx'

### Now Build up using Open APIs

In [8]:
#Now using Korean STT API from Kakao
#Set up the environment
#Requirements: 16000Hz, mono sterio
import requests
import json

In [9]:
url = "https://kakaoi-newtone-openapi.kakao.com/v1/recognize"

In [ ]:
#Part 1: STT
#I had one month free trial for the Open API. (~2021.11.30)
tokens = rest_api_key#put where your app key value is at "rest_api_key"

header = {"Content-Type" : "application/octet-stream",
         "Authorization" : "KakaoAK" + tokens['app_key']}

r = sr.Recognizer()
#sr.Microphone? <- reads the library information
#change the sampling rate as 16k Hz
with sr.Microphone(sample_rate = 16000) as source:
    print("네, 클라우드 원 준비되었습니다. 하명하십시오")
    audio = r.listen(source)

res = requests.post(url, headers = header, data = audio.get_raw_data())

try:
    print(r.recgonize_sphinx(audio)+"라고 인식했습니다")
except sr.UnknownValueError:
    print("미안해요, 제가 좀 더 공부하고 올게요. 다음에는 꼭 도움이 될수 있는 클라우드 원이 되겠습니다")
except sr.RequestError:
    print("죄송해요, 아직 저는 그걸 해낼 능력이 부족하네요. 언젠가는 수행할 수 있도록 최선을 다해 공부하는 클라우드 원이 되겠습니다")

#write audio to a WAV file
with open("microphone-results.wav", "wb") as f:
    f.write(audio.get_wav_data())

print(res.text) #shows what I've said. Final results are saved at "nBest"


In [ ]:
#Tasks that Cloud One can do
#Only in Korean tho
#"오늘 일정 파악해서 주변 맛집 검색해줘" : From Today's task, find the nearby restauratnts with high reviews
#results will be sent to KaKao/Naver/Google 
#Thus, the tokens for access is needed. 
#KaKao = Rest API Key, User Token. 
#Google = Oauth Client ID json
#Naver = Client ID
#I'll put them in auth.py file (has read_auth and update_auth functions)
#codes for this part from ai-creator.tstory/220

from google_auth_authlib.flow import InstalledAppFlow
from auth import read_auth, update_auth
gcreds_filename = read_auth("google")
scopes = ["https://www.googleapis.com/auth/calendar"]
flow = InstalledAppFlow.from_client_secrets_file(gcreds_filename, scopes)
gcreds = flow.run_local_server(port=0)

kakao_tokens = read_auth("kakao")
kakao_tokens = update_auth("kakao", kakao_tokens) #just in case if the token is expired
kakaocreds = {"access_token" : kakao_tokens['access_token']}
kakaoheaders = {"Authorization" : "Bearer " + kakaocreds.get('access_token')}

naver_tokens = read_auth('naver')
naver_creds = {"client_id": naver_tokens['client_id'], 
              "client_secret" : naver_tokens['client_secret']}
naverheaders = {
    "X-Naver-Client-Id" : naver_creds.get('client_id'), 
    "X-Naver-Client-Secret" : ncreds.get('client_secret')}


In [ ]:
#Tasks for Today (Using Google Calendar)
import datetime
import urllib
from googleapiclient.discovery import build

service = build('calendar', 'v3', credentials = gcreds)

##variables
caledar_id = 'primary'
today = datetime.date.today().isoformat()
time_min = today + 'T00:00:00+09:00' #KST = GDT+9 (this is 00:00am in Korea)
time_max = today + 'T23:50:59+09:00' #this is 11:59pm in Korea
max_results = 1 #maximum number of tasks to call. For testing '1' task to call
is_single_events = True #Check if there are repeating tasks
orderby = 'startTime' #Sort the Calendar

##Call the today's tasks
events_result = service.events().list(caledarID = calendar_id, 
                                     timeMin = time_min,
                                     timeMax = time_max,
                                     maxResults = max_results,
                                     singleEvents = is_single_Events,
                                     orderBy = orderby).execute()
events_result.get('items',[])
items = events_result.get('items')
item = items[0]
gsummary = item.get('summary') #name of the task
gcategory, glocation = gsummary[gsummary.index('[')+1 :gsummary.index(']')].split('-')
gevent_url = item.get('htmlLlnk') #getting the google calendar link
gaddress = item.get('location') #getting the location

if ',' in gaddress:
    gaddress = gaddress.split(',')[1].strip()

##Sort the reviews by comment of the blog. 
nparams = 'sort=comment&query=' + enc_location #get the local search results
naver_local_url - "https://openapi.naver.com/v1/search/local.json?" + nparams
res = requests.get(naver_local_url, headers = naverheaders) #search nearby restaurants
##list of 10 restaurants near by
places = res.json().get('items')
###places[0]

##send the result to Kakao Talk (Kinda like Slack or Facebook Message but widely used in Korea)
contents = []
for place in places[:3]:
    ntitle = place.get('title')
    ncategory = place.get('category') #type of cusine
    ntelephone = place.get('telephone')
    nlocation = place.get('address')
    #codes to search the query in Naver
    enc_location - urllib.parse.quote(nlocation + " " + ntitle)
    query = "query=" + enc_location 
    #If the place is a cafe, print coffee cup icon
    #if the place is a restaurant, print food icon
    if '카페' in ncategory:
        image_url = "https://freesvg.org/img/pitr_Coffee_cup_icon.png"
    else: 
        image_url = 'https://freesvg.org/img/bentolunch.png?w=150&h=150&fit=fill'
    #if the phone number exists, print with name
    if ntelephone:
        ntitle = ntitle+"\ntel) "+ntelephone
    #make it fit to the Kakao Talk Template 
    content = {
        "title" : "["+ncategory+"]"+ntite,
        "description": ' '.join(nlocation.split()[1:]),
        "image_url" = image_url,
        "image_width":30, 
        "image_height":30,
        "link" : {
            "web_url":'https://search.naver.com/search.naver?' + query,
            'mobile-web_url' : "https://search.naver.com/search.naver?" + query
        }
    }
    contents.append(content)

##link the calendar to Naver
enc_gaddress = urllib.parse.quote(gaddress)
query = "query=" + enc_gaddress
gaddr_url = "https://search.naver.com/search.naver?" + query

gaddr_url

kakao_template = {
    "object_type":"list",
    "header_title":"'%s' = 맛집추천" %ㅎ녀ㅡㅡㅁ교,
    "header_link":{
        "web_url":gevent_url,
        "mobile_web_url" : gevent_url
    },
    "contents" : contents,
    "buttons"  : [
        {"title": "일정 자세히 보기",
        "link" : {
            "web_url" : gevent_url,
            "mobile_web_url" : gevent_url
        }},
        {"title" : "일정 장소 보기",
        "link": {
            "web_url":gaddr_url,
            "mobile_web_url":gaddr_url
        }}
    ]
}

#Efrom json to string (json.dump> dict -> string(json))
kpayload = {
    "template_object":json.dumps(ktemplate)
}
kpayload

##Now send the text to Kakao
##Custom template url : https://kapi.kakao.com/v2/api/talk/memo/send
##scrap template url : https://kapi.kakao.com/v2/api/talk/memo/scrap/send
kakaotalk_template_url = "https://kapi.kakao.com/v2/api/talk/memo/default/send"
res = requests.post(kakaotalk_template_url, data=kpayload, headers = kakaoheaders)
res.json()

